# Segmenting and Clustering of Neighbourhoods in Toronto

Before we get the data and start chnaging it, let's download all the dependencies that we will need.

In [152]:
import numpy as np
import pandas as pd

#### 1. Read data from the website and transform into a _pandas_ dataframe

In [153]:
# Reading the data and creating a data frame
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df_wiki_can=pd.read_html(url, header=0)[0]

df_wiki_can.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#### 2. Ignore cells with a borough that is Not assigned

In [154]:
# drop all rows in the data frame if value of Borough='Not assigned'
df_wiki_can = df_wiki_can.set_index("Borough")
df_wiki_can = df_wiki_can.drop("Not assigned", axis=0)

df_wiki_can.head()

,Postcode,Neighbourhood
Borough,,
North York,M3A,Parkwoods
North York,M4A,Victoria Village
Downtown Toronto,M5A,Harbourfront
Downtown Toronto,M5A,Regent Park
North York,M6A,Lawrence Heights


In [155]:
# reset index from column Borough
df_wiki_can = df_wiki_can.reset_index()

#### 3. If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [156]:
# Replace Neighbourhood with Borough if Neighbourhood=='Not assigned'
df_wiki_can.Neighbourhood = np.where(df_wiki_can.Neighbourhood.eq('Not assigned'), df_wiki_can.Borough, df_wiki_can.Neighbourhood)

#### 4. If more than one rows with same Postcode, the rows are combined with neighbourhoods seperated with a comma

In [157]:
# grouping rows with same post code values
df_wiki_can = df_wiki_can.groupby(['Postcode','Borough'], sort=False).Neighbourhood.apply(','.join).reset_index(name='Neighbourhood')

In [158]:
df_wiki_can.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [159]:
df_wiki_can.shape

(103, 3)

####  5. Read Latituide and Longitdue data from csv and transform into a _pandas_ data frame

In [160]:
df_geo_can = pd.read_csv('https://cocl.us/Geospatial_data')

print ('Data read into a pandas dataframe!')

Data read into a pandas dataframe!


In [161]:
df_geo_can.shape

(103, 3)

#### 6. Match the headings of both data frames and Merge them

In [162]:
df_geo_can.rename(columns={'Postal Code':'Postcode'}, inplace=True)
df_geo_can.head()#check if changes applied

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### 7. Merge the two data frames and assign results to a new data frame

In [163]:
Toronto_df = df_wiki_can.merge(df_geo_can, on='Postcode')
Toronto_df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


In [164]:
Toronto_df.shape

(103, 5)